In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
import pyvista as pv 
import pandas as pd

In [2]:
cad_data = pd.read_csv('/Users/User/Documents/Project_JEB/JEB_Data/metadata.csv')

In [3]:
cad_data.head(5)

,id,num_vertices,num_faces,volume,surface_area,average_edge_length,genus,max_ver_xdisp,max_ver_ydisp,max_ver_zdisp,...,category,num_tets,mass,download_file,link_name,author,author_id,test_split_0,test_split_1,test_split_2
0,0,42360,84764,307642.703591,73622.298369,1.347077,12.0,0.176923,0.073638,0.288827,...,block,570111,1.375163,FINAL BRACKET AARON WEISSBART 8.9.2013 1930 pa...,ge-bracket-001-1,Aaron Weissbart,aaron.weissbart,False,True,False
1,4,35622,71368,104015.598723,39149.667547,1.048514,32.0,0.687567,0.426113,1.059176,...,beam,431759,0.464950,bracket.STEP,ripple-bracket-1,simon,simon-240,False,False,False
2,6,35186,70532,114996.439753,58499.675375,1.306740,41.0,1.180213,2.789576,3.713256,...,block,314815,0.514034,10813 new GE bracket.IGS,ge-bracket-26,Amartesh Sehgal,amartesh.sehgal-1,False,False,False
3,8,24577,49174,89326.268569,30458.408764,1.130023,6.0,1.044753,0.299482,1.285331,...,beam,293971,0.399288,bracket_15.igs,ge-engine-bracket-15-1,Mandli Peter,mandli.peter,False,False,False
4,9,38322,76720,78486.321312,38243.142004,0.989507,20.0,1.391756,0.360266,1.927059,...,beam,481030,0.350834,EngineBracketModified_PTJ.igs,modified-engine-bracket-2,Penn,penn-1,False,False,False


In [6]:
cad_data = cad_data.drop(columns=['category', 'download_file', 'link_name', 'author', 'author_id', 'test_split_0', 'test_split_1', 'test_split_2'])

In [7]:
cad_data.to_csv('/Users/User/Documents/Project_JEB/JEB_Data/workingdata.csv', index=False)

In [8]:


cad_data['High_Stress_Area'] = 0  

for part_id in cad_data['id']:
    
    print(part_id)
    vtk_file_path = f"/Users/User/Documents/Project_JEB/JEB_Data/SimJEB_volmesh_(vtk)/{part_id}.vtk"
    if part_id <= 308:
        fea_file_path = f"/Users/User/Documents/Project_JEB/JEB_Data/SimJEB_simresults_.csv._firsthalf/SimJEB_simresults_(csv)_firsthalf/{part_id}field.csv"
    else:
        fea_file_path = f"/Users/User/Documents/Project_JEB/JEB_Data/SimJEB_simresults_.csv._secondhalf/SimJEB_simresults_(csv)_secondhalf/{part_id}field.csv"
    
   
    mesh = pv.read(vtk_file_path)

    
    sim_data = pd.read_csv(fea_file_path)
    stress_columns = ['ver_stress', 'hor_stress', 'dia_stress', 'tor_stress']
    sim_data['Stress'] = sim_data[stress_columns].max(axis=1)

   
    stress = sim_data['Stress'].values
    mesh.point_data['Stress'] = stress

    
    stress_threshold = 2500.0  
    critical_areas = mesh.threshold(value=stress_threshold, scalars='Stress')

   
    cad_data.loc[cad_data['id'] == part_id, 'High_Stress_Area'] = critical_areas.n_points


updated_cad_data_path = '/Users/User/Documents/Project_JEB/Results/Data_Analysis/updated_cad_data.csv'
cad_data.to_csv(updated_cad_data_path, index=False)


0
4
6
8
9
10
12
14
15
16
19
20
21
22
23
25
27
28
29
30
33
34
35
38
39
40
50
51
53
55
56
58
59
61
62
63
64
65
66
69
70
71
72
73
76
77
78
79
80
81
82
86
87
91
92
94
95
96
97
98
101
102
106
107
108
112
113
114
115
116
119
120
121
122
123
124
125
127
129
130
131
133
134
135
136
138
139
140
141
142
146
147
148
149
150
152
153
163
164
165
166
167
169
170
171
173
174
175
177
180
181
182
183
185
186
187
191
192
193
194
195
196
197
198
199
200
202
204
205
206
207
208
209
211
213
214
217
220
221
223
225
228
229
230
233
234
235
236
237
238
240
241
242
245
246
247
248
249
252
254
255
256
257
258
259
261
262
266
267
268
269
270
271
275
279
280
281
282
284
285
289
290
291
293
295
296
297
302
306
308
309
310
311
312
316
318
319
320
321
322
323
324
325
329
331
332
335
336
337
339
342
345
348
351
356
357
361
362
364
365
366
369
371
373
374
375
377
378
379
380
382
383
384
386
387
388
389
390
393
394
395
398
400
403
406
407
411
412
413
415
416
417
418
419
420
421
422
424
425
427
428
432
433
434
437
438
4